In [1]:
import spacy
from spacy import displacy
import glob
from evalHelpFunc import callEvaluateExpConDialog, appendSen
from pathlib import Path

import codecs

from texttable import Texttable

#import local .py-Files
from preprocessing import * 
from classRuleStat import * 
from helpFunctions import * 
#import rules
from rConditions import * 
from rFull import * 
from rMinus import * 

In [2]:
#Extra outputs for better information: Set from 0 to 1 for visualization
printText          = 0                            # Print out the actual content of the examplesentence
displayNounPhrases = 1                            # Show the intern Noun_Chunks of the example sentence
displayParseTree   = 1                            # Visualize the ParseTree
displayAnnotations = 0                            # Get additional PoS-Data about every word in the sentence
sentences_path = Path("..") / "Evaluierung" / "AEapaSen" # Path from where to read sentences / "AENonApaSen"
evalSen_path = Path("..") / "Evaluierung" / "resultGbceEvalNonApa.txt" # Path to load results /resultGbceEvalApa

#Load ------------------------------------------------------------------------------------------------------------------
#Load Model ----------------------------------------------
nlp = spacy.load('en_core_web_sm', disable=["ner"])

evalSentences = codecs.open(sentences_path, 'r', 'utf-8')
iSen = 0
for text in evalSentences:
    #print(text)
    if text == " \r\n" or text == "\r\n": 
        continue
    #Preprocess ------------------------------------------------------------------------------------------------------------
    #Check for enumerated aspects


    #delete unimportant noun_chunks
    rPlusMatches, text = handleRPLUSMATCH(text)
    text = whitespaceReduction(text)
    text = deleteSemicolon(text)

    #Check amount of statistics-------
    text = checkAmountOfSentences(text)

    #print Text
    if printText == 1:
        print(text)
    doc = nlp(text)
    if doc[-1].pos_ == "SPACE":
        doc = doc[0:len(doc)-1]
    noun_chunks = modifyNounChunks(doc)
    output = RuleStat(noun_chunks)
    output = output.checkNounChunks() 

    
    #Apply R-Rules: ------------------------------------------------------------------------------------------------------
    output = rMinus(doc, output)
    #Print out additional information: ------------------------------------------------------------------------------------
    if displayNounPhrases == 1:
        print("Noun Phrases: {}".format(output.noun_chunks))
    if displayAnnotations == 1:
        outputSentenceData(doc)
    if displayParseTree == 1:
        print(doc)
        displacy.render(doc, jupyter=True, style='dep', options={'distance': 80})
    #prepositionalModifierDivision(doc, noun_chunks)

    #Apply Extraction Rules: -----------------------------------------------------------------------------------------------
    fullFunctions = [lambda parDoc, parOutput: bagOfWordsSetFull(parDoc, parOutput), 
                           lambda parDoc, parOutput: comparativeAdjective(parDoc, parOutput)]
    findConditionsFunctions = [lambda parDoc, parOutput: bagOfWordsSetCondition(parDoc, parOutput)]
    findAspectFunctions = [lambda parDoc, parOutput: significantExcluder(parDoc, parOutput), 
                           lambda parDoc, parOutput: passiveAuxiliary(parDoc, parOutput)] #add "rootThat(doc, output),"
    for fn in fullFunctions:     # iterate over list of functions, where the current function in the list is referred to as fn
        if output.state["conditions"] or output.state["aspect"] or not output.state["noun_chunks"]:
            break
        else:
            output = fn(doc, output)
    if output.state["noun_chunks"]:
        output = findwhWords(doc, output)

    if not output.state["aspect"]:
        for fn in findAspectFunctions:
            if output.state["aspect"] or not output.state["noun_chunks"]:
                break
            output = fn(doc, output)
    if not output.state["conditions"]:
        for fn in findConditionsFunctions:
            if output.state["conditions"] or not output.state["noun_chunks"]:
                break
            output = fn(doc, output)
    #Check the leftover nounchunks. If they overlap conditions or aspects remove them
    if not output.state["noun_chunks"]:
        output = checkExtractionNeatness(output)

    # If only one nounChunk is left, assign it automatically if possible.
    #if only one nounChunk left, and conditions had been found/ aspect had been found, it must be the other one-------------
    if len(output.noun_chunks) == 1:
        #find out if the last nounChunk should be assigned or not:
        if output.state["conditions"] and output.state["aspect"] == False:
            #add noun_chunk as aspect
            x = output.noun_chunks[0]
            span = doc[x[0]:x[1]+1]
            print("The last nounChunk is:", end = " ")
            for token in span:
                print(token.text, end = " ")
            toggle = input("The last NounChunk is probably experimental data. Do you want to add it? y/n")
            if toggle == "y":
                lastChunk = output.noun_chunks[0]
                output.setconditionBegin(lastChunk[0])
                output.setconditionEnd(lastChunk[1])
                output.deleteNoun_Chunk(lastChunk[0])

        elif output.state["aspect"]:
            #add noun_chunk as condition
            x = output.noun_chunks[0]
            span = doc[x[0]:x[1]+1]
            print("The last nounChunk is:", end = " ")
            for token in span:
                print(token.text, end = " ")
            print()
            if output.state["conditions"]:
                toggle = input("It could be a Condition. Do you want to add it? y/n")
            else:
                toggle = input("The last NounChunk is probably a Condition. Do you want to add it? y/n")
            if toggle == "y":
                lastChunk = output.noun_chunks[0]
                output.setconditionBegin(lastChunk[0])
                output.setconditionEnd(lastChunk[1])
                output.deleteNoun_Chunk(lastChunk[0])

    if output.state["conditions"]:
        output = checkForEnumerations(doc, output)

    #if output.state["aspect"] or output.state["conditions"]:
        #output.printOutput(doc)
    #else:
        #print("No Conditions found, the left over noun_chunks are: '{}'".format(output.noun_chunks))
    conditionSList = output.conditionsToStringList(doc)

    iscorrect, solution, failReason = callEvaluateExpConDialog(text, conditionSList)
    print(iscorrect, solution, failReason)
    if iscorrect:
        appendSen(evalSen_path, iSen, 1, conditionSList, -1, -1)
    else:
        appendSen(evalSen_path, iSen, 0, conditionSList, solution, failReason)
    #appendSen(iSen, output.aspectToString(doc), output.conditionsToString(doc), isCorrect)
    iSen += 1
    #toggle = input("Finished, press any button to go on?")


Noun Phrases: [(0, 1), (3, 7)]
Smartphone distraction correlated significantly with problematic social media use


bagOfWordsSetFull started
Auto-Removed (0, 1) resulting in [(3, 7)]
Auto-[(3, 7)]
Starting checkForEnumerations
Smartphone
distraction
correlated
significantly
problematic
social
media
use
False  
0: 0, ["Smartphone distraction correlated significantly , problematic social media use ]", "", ""

Noun Phrases: [(2, 14)]
There was no relationship between MMN amplitude and either SRS-2 or RBS-R scores


bagOfWordsSetFull started
Keyword: 'between' found
2
[(2, 14)]
Starting checkForEnumerations
CONJUNCTION 'and' FOUND
CONJUNCTION 'either' FOUND
CONJUNCTION 'or' FOUND
MMN
amplitude
SRS-2
RBS
-
R
scores


SystemExit: 

C:\Users\steff\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
